In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # tensorflow INFO and WARNING messages are not printed
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from tqdm import tqdm
import numpy as np
%matplotlib inline

import tensorflow as tf
import keras
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# Vectorize integer sequence
def vectorize_sequence(sequence, dimensions):
    results = np.zeros((len(sequence), dimensions))
    for index,value in enumerate(sequence):
        if max(value) < dimensions:
            results[index, value] = 1
    return results

# Convert label into one-hot format
def one_hot_label(labels, dimensions):
    results = np.zeros((len(labels), dimensions))
    for index,value in enumerate(labels):
        if value < dimensions:
            results[index, value] = 1
    return results

In [3]:
!pip install --upgrade skillsnetwork

In [5]:
import skillsnetwork


In [6]:
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module2/L1/reuters.npz", overwrite=True)

  0%|          | 0/2 [00:00<?, ?it/s]

Saved to '.'


In [7]:
X = np.load("x.npy", allow_pickle=True)
y = np.load ("y.npy", allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

To get the word for a specific index, we can also extract a dictionary of words to index using the following Keras function.

In [8]:
word_to_ind = tf.keras.datasets.reuters.get_word_index(path="reuters_word_index.json")

#Data Wrangling

Since each observation is a list of words that appear in the newswire, the length varies. Hence, we will vectorize the dataset using vectorize_sequence() to ensure that all inputs to our model have the same dimension. Labels are also one-hot encoded with one_hot_label() because classes (news topic) are not ordinal.

In [9]:
dim_x = max([max(sequence) for sequence in X_train])+1
dim_y = max(y_train)+1

X_train_vec = vectorize_sequence(X_train, dim_x)
X_test_vec = vectorize_sequence(X_test, dim_x)
y_train_hot = one_hot_label(y_train, dim_y)
y_test_hot = one_hot_label(y_test, dim_y)

In [10]:
# Create Keras Sequential Model as base model
def create_model(neurons = 10):
    model = Sequential()
    model.add(Dense(neurons, activation='linear'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(46, activation='softmax'))
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
np.random.seed(0)
base_model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=10, epochs=1)

In [12]:
# Get pre-tuned results
base_model.fit(X_train_vec, y_train_hot)
base_score = base_model.score(X_test_vec, y_test_hot)
print("The baseline accuracy is: %.3f" % base_score)

The baseline accuracy is: 0.722


#Randomized Search
##Parameters

As you might already know from performing randomized search on machine learning models, we have to create a dictionary for the hyperparameter values. Let's start by defining the values we want to experiment with! Note that if you would like to test other parameters, they must be defined in the base model as well.

In [13]:
batch_size = [16]
epochs = [1]
neurons = [1, 10, 20, 30]

params = dict(batch_size=batch_size, epochs=epochs, neurons=neurons)
params

{'batch_size': [16], 'epochs': [1, 3, 5], 'neurons': [1, 10, 20, 30]}

In [14]:
#Define and Fit RandomizedSearchCV
search = RandomizedSearchCV(estimator=base_model, param_distributions=params, cv=3)

Now, fit randomized search on X_train_vec and y_train_hot as you would for any other model. Note that this may take a while to run (10+ minutes), especially if there are a lot of parameter combinations, or if the epoch size is big. If you have the resources, you could also switch out RandomizedSearchCV for GridSearchCV to search over every combination of hyperparameters (takes even more time to run).

In [ ]:
search_result = search.fit(X_train_vec, y_train_hot)

In [ ]:
means = search_result.cv_results_['mean_test_score']
stds = search_result.cv_results_['std_test_score']
params = search_result.cv_results_['params']

In [ ]:
print("Best mean cross-validated score: {} using {}".format(round(search_result.best_score_,3), search_result.best_params_))

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print("Mean cross-validated score: {} ({}) using: {}".format(round(mean,3), round(stdev,3), param))

In [ ]:
print("Best test score: %.3f" % search_result.best_estimator_.score(X_test_vec, y_test_hot))

In [ ]:
np.random.seed(0)
base_model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=100, epochs=1)
base_model.fit(X_train_vec, y_train_hot)
base_score = base_model.score(X_test_vec, y_test_hot)
print("The baseline accuracy is: {}".format(base_score))

In [ ]:
optimizer = ['SGD','RMSprop','Adam']
learning_rate = [0.01, 0.1, 1]
dropout_rate = [0.1, 0.3, 0.6, 0.9]
params = dict(optimizer=optimizer, optimizer__learning_rate=learning_rate, dropout_rate = dropout_rate)

In [ ]:
search = RandomizedSearchCV(estimator=base_model, param_distributions=params, cv=3)
search_result = search.fit(X_train_vec, y_train_hot)

In [ ]:
print("Best mean cross-validated score: {} using {}".format(round(search_result.best_score_,3), search_result.best_params_))
print("Best test score: %.3f" % search_result.best_estimator_.score(X_test_vec, y_test_hot))